# Cuaderno 13: Problema de localización de instalaciones (facility location)

Dados un conjunto $I$ de $m$ posibles instalaciones a construir y un conjunto $J$ de $n$ clientes a atender, se requiere decidir cuáles instalaciones construir, y cómo asignar los clientes a estas instalaciones, de tal forma que se cumplan las siguientes 
restricciones:

* para cada cliente $j \in J$, su demanda $d_j$ debe ser atendida en una sola instalación; 
* para cada instalación $i \in I$, la suma de las demandas  de los clientes atendidos por ella no puede
superar su capacidad $u_i$;
* es posible dejar clientes sin atender, por cada cliente no atendido se debe pagar una multa cuyo valor es $M$.

La construcción de la instalación $i \in I$ involucra un costo fijo igual a $f_i$ unidades monetarias. La atención
de un cliente $j \in J$ por una instalación $i \in I$ tiene un costo unitario de $c_{ij}$ unidades monetarias.

Definimos las siguientes variables binarias: 
* $y_i$, $i \in I$, que indican si se construye o no la instalación $i$; 
* $x_{ij}$, $i \in I$, $j \in J$, que registran si el cliente $j$ es atendido por la instalación $i$;
* $z_j$, $j \in J$, que indican si el cliente $j$ es atendido por alguna instalación.

Empleando estas variables, el problema de localización de instalaciones puede ser escrito como el siguiente modelo de programación lineal entera.  

\begin{align*}
\min &\sum_{i \in I} f_i y_i + \sum_{i \in I} \sum_{j \in J} c_{ij} x_{ij} + M \sum_{j \in J} (1 - z_j)\\ 
& \mbox{s.r.}\\
&\sum_{i \in I} x_{ij} = z_j, \quad \forall j \in J, \\
&\sum_{j \in J} d_j x_{ij} \leq u_i y_i, \quad \forall i \in I, \\
& x_{ij}, y_{i}, z_j \in \{0, 1\}, \quad \forall i \in I, j \in J.
\end{align*}


Vamos a implementar este modelo usando la interfaz Python de Gurobi.


Definimos primero los conjuntos $I$, $J$ y los parámetros $f_i$, $c_{ij}$, $M$, $d_j$ y $u_i$:

In [1]:
from gurobipy import *

# Parametros del modelo
# Demandas de los clientes
J, d = multidict(tupledict({1 : 10, 2 : 5,  3 : 7, 4 : 8, 
                 5 : 7,  6 : 6,  7 : 8, 8 : 7}))

# Capacidad y costo fijo de construccion para instalaciones
I, u, f = multidict(tupledict({1 : (25, 2500),
                               2 : (25, 2550),
                               3 : (27, 2850),
                               4 : (26, 2570),
                               5 : (26, 2570)}))

# Costos de atencion
c = tupledict({(1,1) : 25, (1,2): 30, (1,3): 32, (1,4): 35, (1,5): 37, (1,6): 50, (1,7): 23, (1,8) : 48,
               (2,1) : 50, (2,2): 20, (2,3): 22, (2,4): 39, (2,5): 36, (2,6): 20, (2,7): 50, (2,8) : 25,
               (3,1) : 35, (3,2): 25, (3,3): 18, (3,4): 50, (3,5): 45, (3,6): 35, (3,7): 33, (3,8) : 32,
               (4,1) : 25, (4,2): 20, (4,3): 26, (4,4): 20, (4,5): 20, (4,6): 25, (4,7): 37, (4,8) : 35,
               (5,1) : 45, (5,2): 23, (5,3): 20, (5,4): 46, (5,5): 41, (5,6): 24, (5,7): 38, (5,8) : 21})

# Multas
M = 890

print("J = {}".format(J))
print("d = {}".format(d))
print("I = {}".format(I))
print("u = {}".format(u))
print("f = {}".format(f))
print("c = {}".format(c))
print("M = {}".format(M))

J = [1, 2, 3, 4, 5, 6, 7, 8]
d = {1: 10, 2: 5, 3: 7, 4: 8, 5: 7, 6: 6, 7: 8, 8: 7}
I = [1, 2, 3, 4, 5]
u = {1: 25, 2: 25, 3: 27, 4: 26, 5: 26}
f = {1: 2500, 2: 2550, 3: 2850, 4: 2570, 5: 2570}
c = {(1, 1): 25, (1, 2): 30, (1, 3): 32, (1, 4): 35, (1, 5): 37, (1, 6): 50, (1, 7): 23, (1, 8): 48, (2, 1): 50, (2, 2): 20, (2, 3): 22, (2, 4): 39, (2, 5): 36, (2, 6): 20, (2, 7): 50, (2, 8): 25, (3, 1): 35, (3, 2): 25, (3, 3): 18, (3, 4): 50, (3, 5): 45, (3, 6): 35, (3, 7): 33, (3, 8): 32, (4, 1): 25, (4, 2): 20, (4, 3): 26, (4, 4): 20, (4, 5): 20, (4, 6): 25, (4, 7): 37, (4, 8): 35, (5, 1): 45, (5, 2): 23, (5, 3): 20, (5, 4): 46, (5, 5): 41, (5, 6): 24, (5, 7): 38, (5, 8): 21}
M = 890


Definimos ahora el objeto modelo y las variables del modelo:

In [2]:
m = Model('facility-location')

# Asignacion instalaciones-clientes:
x = m.addVars(I, J, vtype = GRB.BINARY, name="x")

# Construccion de instalaciones
y = m.addVars(I, vtype = GRB.BINARY, name="y")

# Atencion de clientes
z = m.addVars(J, vtype = GRB.BINARY, name="z")


--------------------------------------------
--------------------------------------------

Using license file /Users/ltorres/gurobi.lic
Academic license - for non-commercial use only


Construimos la función objetivo a partir de sus tres términos:

In [3]:
# costos fijos de construccion
costo_construccion = y.prod(f, '*')

# costos de atencion a clientes
costo_atencion = x.prod(c, '*', '*')

# multas por no atencion
multas = M*(len(J) - z.sum('*'))

m.setObjective(costo_construccion + costo_atencion + multas, GRB.MINIMIZE)

Finalmente, implementamos las restricciones del modelo:
1. Cada cliente $j$ es atendido por una sola instalación, si $z_j = 1$; o no es atendido en ninguna instalación, si $z_j = 0$.

In [4]:
m.addConstrs((x.sum('*', j) == z[j] for j in J), "asig") 

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>}

2. Asignar clientes únicamente a instalaciones que son construidas, y respetar la capacidad de las instalaciones.

In [5]:
m.addConstrs((quicksum(d[j] * x[i,j] for j in J) <= u[i]*y[i] for i in I), "capacidad")

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

Optimizar el modelo:

In [6]:
m.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 13 rows, 53 columns and 93 nonzeros
Model fingerprint: 0x6ff1a1e3
Variable types: 0 continuous, 53 integer (53 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [2e+01, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 7120.0000000
Presolve removed 0 rows and 8 columns
Presolve time: 0.01s
Presolved: 13 rows, 45 columns, 85 nonzeros
Variable types: 0 continuous, 45 integer (45 binary)

Root relaxation: objective 5.791846e+03, 12 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5791.84615    0    3 7120.00000 5791.84615  18.7%     -    0s
H    0     0                    7045.0000000 5791.84615  17.8%     -    0s
H    0     0                    6215.0000000 5791.84615  6.81%     

Mostrar la solución: instalaciones a construir, asignación de clientes a instalaciones y clientes no atendidos.

In [7]:
# Instalaciones a construir
for i in I:
    if y[i].x >= 0.99:
        print("Se construye la instalacion {}.".format(i))
        
# Asignacion de clientes a instalaciones
for i in I:
    for j in J:
        if x[i, j].x >= 0.99:
            print("Cliente {} atendido por la instalacion {}.".format(j, i))
            
# Clientes no atendidos
for j in J:
    if z[j].x <= 0.01:
        print("Cliente {} no es atendido.".format(j))

Se construye la instalacion 1.
Se construye la instalacion 2.
Cliente 1 atendido por la instalacion 1.
Cliente 5 atendido por la instalacion 1.
Cliente 7 atendido por la instalacion 1.
Cliente 2 atendido por la instalacion 2.
Cliente 3 atendido por la instalacion 2.
Cliente 6 atendido por la instalacion 2.
Cliente 8 atendido por la instalacion 2.
Cliente 4 no es atendido.


## Código completo

Se reproduce a continuación el código completo del modelo anterior.

In [16]:
# Curso de implementacion de programas lineales enteros
# Ejemplo: Modelo de localización de instalaciones (facility location)
# EPN (2020)

from gurobipy import *
try:
    # Parametros del modelo
    # Demandas de los clientes
    J, d = multidict(tupledict({1 : 10, 2 : 5,  3 : 7, 4 : 8, 
                 5 : 7,  6 : 6,  7 : 8, 8 : 7}))

    # Capacidad y costo fijo de construccion para instalaciones
    I, u, f = multidict(tupledict({1 : (25, 2500),
                                   2 : (25, 2550),
                                   3 : (27, 2850),
                                   4 : (26, 2570),
                                   5 : (26, 2570)}))

    # Costos de atencion
    c = tupledict({(1,1) : 25, (1,2): 30, (1,3): 32, (1,4): 35, (1,5): 37, (1,6): 50, (1,7): 23, (1,8) : 48,
                   (2,1) : 50, (2,2): 20, (2,3): 22, (2,4): 39, (2,5): 36, (2,6): 20, (2,7): 50, (2,8) : 25,
                   (3,1) : 35, (3,2): 25, (3,3): 18, (3,4): 50, (3,5): 45, (3,6): 35, (3,7): 33, (3,8) : 32,
                   (4,1) : 25, (4,2): 20, (4,3): 26, (4,4): 20, (4,5): 20, (4,6): 25, (4,7): 37, (4,8) : 35,
                   (5,1) : 45, (5,2): 23, (5,3): 20, (5,4): 46, (5,5): 41, (5,6): 24, (5,7): 38, (5,8) : 21})

    # Multas
    M = 890
    
    # Crear el objeto modelo
    m = Model('facility-location')

    # Asignacion instalaciones-clientes:
    x = m.addVars(I, J, vtype = GRB.BINARY, name="x")

    # Construccion de instalaciones
    y = m.addVars(I, vtype = GRB.BINARY, name="y")

    # Atencion de clientes
    z = m.addVars(J, vtype = GRB.BINARY, name="z")

    # Funcion objetivo
    # costos fijos de construccion
    costo_construccion = y.prod(f, '*')

    # costos de atencion a clientes
    costo_atencion = x.prod(c, '*', '*')

    # multas por no atencion
    multas = M*(len(J) - z.sum('*'))

    m.setObjective(costo_construccion + costo_atencion + multas, GRB.MINIMIZE)
    
    # Restricciones
    # Si un cliente es atendido, es asignado a una sola instalación
    m.addConstrs((x.sum('*', j) == z[j] for j in J), "asig") 

    # Los clientes son asignados solamente a instalaciones construidas, 
    # respetando su capacidad
    m.addConstrs((quicksum(d[j] * x[i,j] for j in J) <= u[i]*y[i] for i in I), "capacidad")
    
    # Resolver el modelo
    m.optimize()
    
    # Mostrar la solución
    # Instalaciones a construir
    for i in I:
        if y[i].x >= 0.99:
            print("Se construye la instalacion {}.".format(i))
        
    # Asignación de clientes a instalaciones
    for i in I:
        for j in J:
            if x[i, j].x >= 0.99:
                print("Cliente {} atendido por la instalacion {}.".format(j, i))
            
    # Clientes no atendidos
    for j in J:
        if z[j].x <= 0.01:
            print("Cliente {} no es atendido.".format(j))    
except GurobiError as e:
    print('Se produjo un error de Gurobi: codigo: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 13 rows, 53 columns and 93 nonzeros
Model fingerprint: 0x6ff1a1e3
Variable types: 0 continuous, 53 integer (53 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [2e+01, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 7120.0000000
Presolve removed 0 rows and 8 columns
Presolve time: 0.00s
Presolved: 13 rows, 45 columns, 85 nonzeros
Variable types: 0 continuous, 45 integer (45 binary)

Root relaxation: objective 5.791846e+03, 12 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5791.84615    0    3 7120.00000 5791.84615  18.7%     -    0s
H    0     0                    7045.0000000 5791.84615  17.8%     -    0s
H    0     0                    6215.0000000 5791.84615  6.81%     